In [1]:
#FOR OVERFIT #######################################################################
####################################################################################
####################################################################################
import torch
import numpy as np
from dataset import ShapeNetPartDataset
from model import MultiSacleUNet
import h5py

class_name = np.genfromtxt('all_object_categories.txt',dtype='U')[:,0]

dataset_path = "shapenet_prepared.h5"
f = h5py.File(dataset_path,'r')
x_test = f['x_test'][0]
y_test = f['y_test'][0]
s_test = f['s_test'][0]
p_test = f['p_test'][0]

# give the location of your pre-trained model
ckpt = "C:/Users/aorhu/Masaüstü/ML3DG_Project/3d-segmentation-in-2d/runs/temp_model_1/model.ckpt"
model = MultiSacleUNet()
model.load_state_dict(torch.load(ckpt, map_location='cpu'))
model.eval()

#holds the class specific labels, will 12,13,14 in overfit case
class_label_region = np.zeros((1,2),dtype=np.int64)

label_min = s_test.min()
label_max = s_test.max()

# this double indexing is from the test script for multiple examples, not necessary to change since the rest works according to this
class_label_region[0,0] = label_min
class_label_region[0,1] = label_max

# testing for multiple examples normally reads from the h5 files in these lines but it's not necessary for a single example
x_set = x_test # point cloud position in 3D
y_set = y_test # point cloud shape class
s_set = s_test # point cloud segments
p_set = p_test # 2D position
pre_set = np.zeros((1,2048,1))

In [2]:
#need to have shapenet_prepared.h5 file ready under same directory with this file
test_dataset = ShapeNetPartDataset(
    path='shapenet_prepared.h5',
    split='overfit'
    )
pre_test = np.zeros_like(s_test)

In [3]:
pre_sample = np.zeros_like(pre_test)
#getting class specific min and max labels, it's actually same with the lines 149-150 in overfit case. 
#I'll change it once we figure out proper training
label_min = int(class_label_region[0,0])
label_max = int(class_label_region[0,1]+1)

In [4]:
#reading the 0th sample from the dataset, which is the overfitted example in our case
input_tensor = torch.tensor(test_dataset.__getitem__(0)["3d_points"])
#uncomment this to view whole tensor(things get slow)
#torch.set_printoptions(profile="default")
#print(input_tensor)
print(input_tensor.shape)
input_tensor = input_tensor[None,:]
print(input_tensor.shape)

torch.Size([3, 256, 256])
torch.Size([1, 3, 256, 256])


C:\Users\aorhu\AppData\Local\Temp\ipykernel_12204\3956704507.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(test_dataset.__getitem__(0)["3d_points"])


In [5]:
#putting the input through model
pre_image = model(input_tensor)[0]
print(pre_image.shape)
#torch.set_printoptions(profile="full")
print(pre_image)

torch.Size([51, 256, 256])
tensor([[[-0., -0., -0.,  ..., -0., 1., -0.],
         [-0., -0., -0.,  ..., 1., -0., -0.],
         [-0., -0., -0.,  ..., -0., -0., -0.],
         ...,
         [-0., 1., -0.,  ..., -0., 0., -0.],
         [1., -0., -0.,  ..., 0., -0., 1.],
         [-0., -0., -0.,  ..., -0., 1., -0.]],

        [[-0., -0., -0.,  ..., 1., -0., -0.],
         [-0., -0., -0.,  ..., -0., -0., -0.],
         [-0., -0., -0.,  ..., -0., -0., -0.],
         ...,
         [1., -0., -0.,  ..., -0., -0., 1.],
         [-0., -0., -0.,  ..., -0., 1., -0.],
         [-0., -0., -0.,  ..., 1., -0., -0.]],

        [[-0., -0., -0.,  ..., -0., -0., -0.],
         [-0., -0., -0.,  ..., -0., -0., -0.],
         [-0., -0., -0.,  ..., -0., -0., -0.],
         ...,
         [-0., -0., -0.,  ..., -0., 1., -0.],
         [-0., -0., -0.,  ..., 1., -0., -0.],
         [-0., -0., -0.,  ..., -0., -0., -0.]],

        ...,

        [[-0., -0., 1.,  ..., -0., -0., 0.],
         [-0., 1., -0.,  ..., -0., 

In [6]:
#applying argmax to logits 
print("before argmax : ",pre_image.shape)
#pre_image = pre_image[:,:,label_min:label_max].argmax(1)+label_min
pre_image = torch.argmax(pre_image,dim=0) + label_min
print("after argmax : ",pre_image.shape)
#torch.set_printoptions(profile="full")
print(pre_image)

before argmax :  torch.Size([51, 256, 256])
after argmax :  torch.Size([256, 256])
tensor([[62, 61, 60,  ..., 13, 12, 62],
        [61, 60, 59,  ..., 12, 62, 61],
        [60, 59, 58,  ..., 62, 61, 60],
        ...,
        [13, 12, 62,  ..., 15, 14, 13],
        [12, 62, 61,  ..., 14, 13, 12],
        [62, 61, 60,  ..., 13, 12, 62]])


In [7]:
#these lines are also for the testing with multiple samples but no need to change for now
pre_sample = pre_image[p_test[:,0],p_test[:,1]]
print(pre_sample.shape)
pre_test[0] = pre_sample[:,None][0]
pre_set  = pre_sample[:,None][0]


torch.Size([2048])


In [8]:
pre_sample

tensor([37, 43, 23,  ..., 19, 48, 58])

In [9]:
#iou calculation, will only calculate one class which is chair for overfit example
iou_shape = np.zeros(len(s_test))
#for idx_sample,pos,obj_class in zip(range(0,len(p_test)),p_test,y_test):

for _pre,gt_sample in zip(pre_test,s_test):
    label_min = int(class_label_region[0,0])
    label_max = int(class_label_region[0,1]+1)

    iou_list = []
    # % for each segment, calculate iou
    for i_class in range(label_min,label_max):
        # break
        tp = np.sum( (_pre == i_class) * (gt_sample == i_class) )
        fp = np.sum( (_pre == i_class) * (gt_sample != i_class) )
        fn = np.sum( (_pre != i_class) * (gt_sample == i_class) )

        # % if current segment exists then count the iou
        iou = (tp+1e-12) / (tp+fp+fn+1e-12)

        iou_list.append(iou)

    iou_shape[0] = np.mean(iou_list)

In [10]:
print( 'iou_instance =', iou_shape.mean())


iou_instance = 0.0003255208333334961


In [11]:
#%% calculate iou for each class
#iou calculation for all classes
iou_class = np.zeros(16)
for obj_class in range(16):
    iou_obj_class = iou_shape[y_test==obj_class]
    iou_class[obj_class] = iou_obj_class.mean()
print( 'iou_class =', iou_class.mean())

for obj_class in range(16):
    print('class',obj_class,', class name:',class_name[obj_class],",iou=",iou_class[obj_class])

iou_class = nan
class 0 , class name: Airplane ,iou= nan
class 1 , class name: Bag ,iou= nan
class 2 , class name: Cap ,iou= nan
class 3 , class name: Car ,iou= nan
class 4 , class name: Chair ,iou= 0.0003255208333334961
class 5 , class name: Earphone ,iou= nan
class 6 , class name: Guitar ,iou= nan
class 7 , class name: Knife ,iou= nan
class 8 , class name: Lamp ,iou= nan
class 9 , class name: Laptop ,iou= nan
class 10 , class name: Motorbike ,iou= nan
class 11 , class name: Mug ,iou= nan
class 12 , class name: Pistol ,iou= nan
class 13 , class name: Rocket ,iou= nan
class 14 , class name: Skateboard ,iou= nan
class 15 , class name: Table ,iou= nan


C:\Users\aorhu\AppData\Local\Temp\ipykernel_12204\502392668.py:6: RuntimeWarning: Mean of empty slice.
  iou_class[obj_class] = iou_obj_class.mean()
c:\Users\aorhu\anaconda3\envs\3d-seg-in-2d\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [12]:
# %%
#VISUALIZATION
#visualizatin normally will be a different script by itself but just for this overfit sample visualization
#the code is copied to here so some repetition of the code might be seen, not a problem, will be fixed
# install plotly first then it might give an error while plotting, something about "nb", only need to update some package then restart, it fixed for me
import numpy as np
import seaborn as sns
import plotly.graph_objects as go

current_palette = sns.color_palette('bright',10)
#"all_object_categories.txt" should also be under the same directory with this script
class_name = np.genfromtxt('all_object_categories.txt',dtype='U')[:,0]

In [13]:
label_min = s_test.min()
label_max = s_test.max()
x_pt = x_set
s_pt = s_set-label_min
pre_pt = pre_sample-label_min

In [14]:
pre_pt

tensor([25, 31, 11,  ...,  7, 36, 46])

In [15]:
#for ground truth
fig = go.Figure()
for i_seg in range(label_max - label_min +1):
    idxs = np.where(s_pt == i_seg)[0]
    color = current_palette.as_hex()[i_seg]
    fig.add_trace(go.Scatter3d(x=x_pt[idxs,0], y=x_pt[idxs,1], z=x_pt[idxs,2],
                                   mode='markers',
                                   marker = dict(color = color )))
    
fig.show()

In [16]:
#for prediction
fig = go.Figure()
for i_seg in range(label_max - label_min +1):
    idxs = np.where(pre_pt == i_seg)[0]
    color = current_palette.as_hex()[i_seg]
    fig.add_trace(go.Scatter3d(x=x_pt[idxs,0], y=x_pt[idxs,1], z=x_pt[idxs,2],
                                   mode='markers',
                                   marker = dict(color = color )))
    
fig.show()